In [1]:
import dynamiqs as dq
import jax.numpy as jnp

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML, display

In [ ]:
## Task 2.1 - Lab frame simulation
# Hilbert space truncations
na = 20  # Memory mode a - encodes the logical cat qubit
nb = 5   # Buffer mode b - mediates 2-photon dissipation to stabilize the cat state

# Annihilation operators in the composite Hilbert space
a = dq.tensor(dq.destroy(na), dq.eye(nb))   # Memory mode a annihilation operator
b = dq.tensor(dq.eye(na), dq.destroy(nb))   # Buffer mode b annihilation operator

# Both modes start in a vacuum 
psi0 = dq.tensor(dq.fock(na, 0), dq.fock(nb, 0))

# Using A100, so double is supported. We used Clemson's Palmetto 2 Supercomputer
dq.set_precision('double') 

# Define constants
w_a0 = 5.26       # [GHz]
w_b0 = 7.70       # [GHz]
psi_a = 0.06      # [-]
psi_b = 0.29      # [-]
E_j = 42.76       # [GHz]
DE_j = 0.47       # [GHz]
w_d = 7.623       # [GHz]
w_p = 2.891       # [GHz]
e_d = -3.815e-3   # [GHz]
e_p = 0.122       # [rad]
kappa_a = 9.3e-6  # [GHz]
kappa_b = 2.6e-3  # [GHz]

# Initialize the time array
T = 4
tsave = jnp.linspace(0, T, 100)

# Returns the Asymmetrically Threaded SQUID Hamiltonian
def ATS(t):
    e_t = e_p * jnp.cos(w_p * t)
    A = psi_a * (a + a.dag()) + psi_b * (b + b.dag())
    term1 = -2*E_j * jnp.sin(e_t) * jnp.sin(dq.to_jax(A)) 
    term2 = 2 * DE_j * jnp.cos(e_t) * jnp.cos(dq.to_jax(A))
    ATS = term1 + term2
    return ATS 

# Returns the 1-photon drive Hamiltonian
def d(t):
    d = 2 * e_d * jnp.cos(w_d * t)
    return d * (b + dq.dag(b))

# Returns the total Hamiltonian
def get_H(t):
    H_0 = w_a0 * (dq.dag(a) @ a) + w_b0 * (dq.dag(b) @ b)
    H_d = d(t)
    H_ATS = ATS(t)
    return H_0 + H_ATS + H_d

# The Hamiltonian depends explicitly on t, so we need timecallable()
H = dq.timecallable(get_H)

# Typical 2-mode loss operator
loss_ops = [jnp.sqrt(kappa_a) * a, jnp.sqrt(kappa_b) * b]

# Results from solving the Lindblad master equation
res = dq.mesolve(H, loss_ops, psi0, tsave)

# Animation plotting
fig_w, ax_w = plt.subplots(1, 1, figsize=(6, 6))
def update(frame):
    ax_w.cla()                                # Clear the axis
    rho_a = dq.ptrace(res.states[frame], 0)   # Partial trace over the buffer mode to extract the reduced state of the memory mode
    dq.plot.wigner(rho_a, ax=ax_w)            # Plots the Wigner function for the reduced state above, rho_a
    ax_w.set_title(f"Mode a Wigner Function\nTime = {tsave[frame]:.2f}")   # Sets the title of the animation

ani = FuncAnimation(fig_w, update, frames=len(tsave), repeat=False)   # Creates the animation
gif_filename = 'Task 2.1 Wigner Function Evolution.gif'               # Animation file name
ani.save(gif_filename, writer=PillowWriter(fps=len(tsave) / 4))       # Saves the animation to the same directory that this file is in
plt.close(fig_w)                                                      # Closes the file in matplotlib to free up memory 
display(HTML(f'<img src="{gif_filename}">'))                          # Displays the animation file

In [ ]:
## Task 2.2 - Rotated-displaced frame simulation